## Develop summary table of all Sensors in LoRa Database

In [17]:
from tempfile import TemporaryDirectory
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone

pd.options.display.max_rows = 200

In [9]:
gtw_file = '~/gateways.tsv'
df = pd.read_csv(gtw_file, sep='\t', parse_dates=['ts'])
df.head()

,dev_id,ts,ts_day,ts_hour,counter,gateway,snr,rssi,data_rate
0,boat-lt2-a726,2021-09-23 00:00:26,2021-09-23,2021-09-23 00:00,1003,packetbroker,-10.50,-108,SF10BW125.0
1,erslite-marquam-56a5,2021-09-23 00:00:53,2021-09-23,2021-09-23 00:00,184,packetbroker,10.25,-33,SF7BW125.0
2,ldds75-a84041281182a87f,2021-09-23 00:00:55,2021-09-23,2021-09-23 00:00,367,packetbroker,9.00,-72,SF7BW125.0
3,ersco2-msb-admin-26db,2021-09-23 00:00:59,2021-09-23,2021-09-23 00:00,20103,msb-admin-a840411eb4f84150,7.50,-29,SF7BW125.0
4,ersco2-msb-admin-26db,2021-09-23 00:00:59,2021-09-23,2021-09-23 00:00,20103,packetbroker,-3.20,-121,SF7BW125.0


In [10]:
tz_ak = timezone('US/Alaska')
ts_start = (datetime.now(tz_ak) - timedelta(days=1)).replace(tzinfo=None)
ts_start

datetime.datetime(2021, 10, 22, 9, 53, 36, 730906)

In [11]:
df1d = df.query('ts >= @ts_start')

In [12]:
dfs = df1d.groupby('dev_id').agg(
    {
        'data_rate': 'last',
        'counter': ['first', 'last']
    }
)
dfs.columns = ['data_rate', 'counter_first', 'counter_last']
dfs['total_rdg'] = dfs.counter_last - dfs.counter_first + 1
dfs.head()

,data_rate,counter_first,counter_last,total_rdg
dev_id,,,,
boat-lt2-a725,SF10BW125.0,5411,5476,66
boat-lt2-a726,SF10BW125.0,2270,2557,288
boat-lt2-a728,SF10BW125.0,469,217,-251
boat-lt2-a72b,SF10BW125.0,9473,9760,288
boat-lt2-a72d,SF10BW125.0,3744,4031,288


In [13]:
df_rcvd = df1d[['dev_id', 'counter']].drop_duplicates()
df_rcvd = df_rcvd.groupby('dev_id').count()
df_rcvd.columns = ['rcvd']
df_rcvd.head()

,rcvd
dev_id,
boat-lt2-a725,3
boat-lt2-a726,279
boat-lt2-a728,288
boat-lt2-a72b,282
boat-lt2-a72d,279


In [14]:
df_final = dfs.join(df_rcvd)
df_final['success_pct'] = df_final.rcvd / df_final.total_rdg
df_display = df_final[['data_rate', 'success_pct']].copy()
df_display['data_rate'] = df_display.data_rate.str.replace('.0', '', regex=False)
df_display.columns = ['Data Rate', 'Success %']
df_display.index.name = 'Dev ID'
s2 = df_display.style.applymap(lambda v: 'color:red;' if v < 0.9 else None, subset=['Success %'])
s2

,Data Rate,Success %
Dev ID,,
boat-lt2-a725,SF10BW125,0.045455
boat-lt2-a726,SF10BW125,0.968750
boat-lt2-a728,SF10BW125,-1.147410
boat-lt2-a72b,SF10BW125,0.979167
boat-lt2-a72d,SF10BW125,0.968750
boat-lt2-a72f,SF10BW125,-0.757895
boat-lt2-a84041552182436a,SF10BW125,0.975694
boat-lt2-a840417f8182436e,SF7BW125,-0.029210
boat-lt2-phoenix-swd,SF7BW125,0.954704
